## Ejercicio 4: Convertir datos financieros estructurados en un informe narrativo

Partiendo de un dataframe con datos financieros, usaremos la API de OpenAI para generar un reporte en lenguaje natural

In [2]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
API_VERSION = userdata.get('OPENAI_API_VERSION')
AZURE_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')

#print(type(OPENAI_API_KEY),OPENAI_API_KEY)
#print(type(API_VERSION),API_VERSION)
#print(type(AZURE_ENDPOINT),AZURE_ENDPOINT)

In [3]:
import pandas as pd
import numpy as np
from openai import AzureOpenAI
import json

##### Establecemos el valor de la API key de OpenAI. En una aplicación real, no podemos exponer esta API key en el código, y la cogeríamos con una variable de entorno mediante un os.getenv()

In [4]:
np.random.seed(42)
num_filas = 100  # Número de registros

# Generamos fechas aleatorias entre el 01/01/2023 y el 31/12/2024
fechas = pd.to_datetime(
    np.random.choice(pd.date_range("2023-01-01", "2024-12-31"), size=num_filas)
)

# Generamos datos simulados
ingresos = np.random.randint(100000, 500000, size=num_filas)
gastos = np.random.randint(80000, 400000, size=num_filas)
balance = ingresos - gastos
numero_clientes = np.random.randint(1000, 10000, size=num_filas)
costos_operativos = np.random.randint(50000, 150000, size=num_filas)

# Creamos el DataFrame
df_financiero = pd.DataFrame({
    "Fecha": fechas,
    "Ingresos": ingresos,
    "Gastos": gastos,
    "Balance": balance,
    "Numero_de_clientes": numero_clientes,
    "Costos_operativos": costos_operativos
})

df_financiero = df_financiero.sort_values("Fecha").reset_index(drop=True)

# Guardamos el DataFrame en un archivo CSV
df_financiero.to_csv("financial_data.csv", index=False, encoding="utf-8")
print("Archivo 'financial_data.csv' generado exitosamente.\n")

Archivo 'financial_data.csv' generado exitosamente.



##### Leemos el csv como dataframe de pandas, y lo convertimos en json

In [8]:
df_report = pd.read_csv("financial_data.csv", parse_dates=["Fecha"])
print("df_report shape:", df_report.shape)
display(df_report.head())

datos_json = df_report.to_json(orient="records", date_format="iso", force_ascii=False)
#print(datos_json)

df_report shape: (100, 6)


,Fecha,Ingresos,Gastos,Balance,Numero_de_clientes,Costos_operativos
0,2023-01-02,164044,213121,-49077,8206,85247
1,2023-01-14,210687,299686,-88999,6442,122789
2,2023-01-15,435674,337186,98488,4854,110890
3,2023-01-21,264231,394877,-130646,7015,108335
4,2023-01-21,345310,317714,27596,4394,109638


##### Definimos el prompt

In [13]:
#Primero del digo el ROL
#Luego le digo lo que quiero que haga enumerado
#Despues del punto 6, le digo el estilo.
# Normalmente se crea un diccionario de tonos de lenguaje, de conclusiones a obtener, etc
prompt_informe = (
    "Eres un analista financiero con amplia experiencia en la elaboración de informes detallados. "
    "A partir de los siguientes datos financieros, debes generar un informe narrativo extenso que contenga las siguientes secciones:\n\n"
    "1. **Resumen Ejecutivo:** Resume la situación general y las tendencias globales observadas en los datos. "
    "Destaca los periodos de mayor rendimiento y las áreas de oportunidad.\n\n"
    "2. **Análisis de Ingresos y Gastos:** Describe en detalle cómo varían los ingresos y gastos a lo largo del tiempo. "
    "Identifica periodos con altas discrepancias y analiza posibles causas de dichas variaciones.\n\n"
    "3. **Evaluación del Balance:** Explica la evolución del balance resultante de la diferencia entre ingresos y gastos. "
    "Relaciona estos cambios con el comportamiento de los otros indicadores.\n\n"
    "4. **Impacto en la Base de Clientes:** Analiza cómo el número de clientes podría influir en los resultados financieros. "
    "Comenta sobre la posible relación entre el crecimiento o decrecimiento de clientes y el rendimiento financiero.\n\n"
    "5. **Costos Operativos y Eficiencia:** Examina los costos operativos y su impacto en la eficiencia global del banco. "
    "Incluye observaciones sobre la optimización de procesos y la reducción de gastos innecesarios.\n\n"
    "6. **Conclusiones y Recomendaciones:** Concluye el informe con un resumen de los hallazgos y proporciona recomendaciones "
    "estratégicas basadas en el análisis realizado.\n\n"
    "Utiliza un lenguaje técnico pero claro, adecuado para un público de profesionales financieros."
    "Asegúrate de que el informe sea coherente, estructurado y que cada sección se conecte lógicamente con la siguiente.\n\n"
    "A continuación, se presentan los datos financieros en formato JSON:\n"
    f"{datos_json}\n\n"
    "Informe narrativo:"
)

##### Creamos el informe

In [14]:
from IPython.display import Markdown

client = AzureOpenAI(
            api_key = OPENAI_API_KEY,
            api_version=API_VERSION,
            azure_endpoint= AZURE_ENDPOINT# Este endpoint corresponde a la región france-central
)

response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt_informe}],
        temperature=0.5,
        max_tokens=2000,
    )

informe_narrativo = response.choices[0].message.content.strip()
print("Informe narrativo generado:\n")
display(Markdown(informe_narrativo))
#si no funciona, descomentar esta línea:
#print(informe_narrativo)

Informe narrativo generado:



# Informe Financiero Detallado

## 1. **Resumen Ejecutivo**

El análisis de los datos financieros proporcionados para el periodo comprendido entre enero de 2023 y diciembre de 2024 revela un comportamiento heterogéneo en términos de ingresos, gastos y balances. A lo largo del periodo, se observan variaciones significativas en el rendimiento financiero, con fluctuaciones marcadas tanto en los ingresos como en los gastos. Los balances resultantes muestran periodos de alta rentabilidad intercalados con pérdidas notables, lo que sugiere una dinámica operativa sujeta a factores externos y posibles ineficiencias internas.

Los periodos más destacados por su rendimiento positivo incluyen fechas como el **10 de febrero de 2023**, con un balance de $332,298, y el **28 de septiembre de 2023**, con un balance excepcional de $400,966. Por otro lado, los periodos de mayor pérdida incluyen el **30 de diciembre de 2024**, con un balance negativo de $-96,638, y el **20 de diciembre de 2024**, con una pérdida de $-258,545.

En términos de áreas de oportunidad, se identificaron patrones de altos costos operativos en diversos periodos, lo que impactó negativamente en la eficiencia y los márgenes de rentabilidad. Además, se observó una relación inconsistente entre el número de clientes y los resultados financieros, lo que sugiere posibles problemas en la gestión del portafolio de clientes o en la optimización de ingresos por cliente.

---

## 2. **Análisis de Ingresos y Gastos**

### Ingresos
Los ingresos presentan una tendencia fluctuante, con picos significativos en fechas específicas. Por ejemplo:
- El **22 de enero de 2023**, los ingresos alcanzaron $470,210, impulsando un balance positivo de $291,704.
- El **28 de septiembre de 2023**, los ingresos fueron de $487,261, uno de los valores más altos del periodo analizado.

Sin embargo, en algunos periodos, los ingresos disminuyeron considerablemente, como el **5 de noviembre de 2023**, con ingresos de solo $130,535, lo que coincidió con un balance negativo de $-206,891.

### Gastos
Los gastos también muestran una variación considerable, con ciertos picos que superan los ingresos y generan balances negativos significativos. Por ejemplo:
- El **19 de julio de 2024**, los gastos ascendieron a $363,076, frente a ingresos de $102,693, resultando en un balance negativo de $-260,383.
- El **20 de diciembre de 2024**, los gastos alcanzaron $380,504, mientras que los ingresos fueron de solo $121,959, lo que generó una pérdida de $-258,545.

En contraste, en periodos como el **28 de septiembre de 2023**, los gastos estuvieron controlados ($86,295), lo que permitió un balance positivo de $400,966.

### Discrepancias
Las discrepancias más notables entre ingresos y gastos ocurrieron en periodos de altos costos operativos o ingresos anormalmente bajos. Estas discrepancias pueden atribuirse a factores como estacionalidad, gestión ineficiente de recursos o eventos externos que afectaron la generación de ingresos.

---

## 3. **Evaluación del Balance**

El balance, definido como la diferencia entre ingresos y gastos, refleja la salud financiera de la organización. Durante el periodo analizado, los balances positivos más destacados ocurrieron en:
- **28 de septiembre de 2023** ($400,966).
- **23 de mayo de 2024** ($314,461).
- **21 de enero de 2024** ($283,965).

Por otro lado, los balances negativos más significativos ocurrieron en:
- **19 de julio de 2024** ($-260,383).
- **20 de diciembre de 2024** ($-258,545).
- **5 de noviembre de 2023** ($-206,891).

En general, los balances positivos tienden a coincidir con una combinación de altos ingresos y control de gastos, mientras que los balances negativos suelen estar asociados a picos de gastos desproporcionados o ingresos insuficientes.

---

## 4. **Impacto en la Base de Clientes**

El número de clientes muestra una relación no lineal con los resultados financieros. Aunque un mayor número de clientes podría sugerir mayores ingresos, esto no siempre se traduce en balances positivos. Por ejemplo:
- El **10 de febrero de 2023**, con 8,509 clientes, se alcanzó un balance positivo de $332,298.
- Sin embargo, el **5 de noviembre de 2023**, con 4,354 clientes, se registró un balance negativo de $-206,891.

En algunos casos, periodos con pocos clientes lograron balances positivos debido al control de costos operativos. Por ejemplo, el **23 de mayo de 2024**, con solo 1,154 clientes, se obtuvo un balance de $314,461 gracias a ingresos elevados ($424,767) y costos operativos moderados.

Esto sugiere que la rentabilidad no depende exclusivamente del volumen de clientes, sino también de la capacidad de maximizar los ingresos por cliente y gestionar eficientemente los costos.

---

## 5. **Costos Operativos y Eficiencia**

Los costos operativos representan un componente clave en la eficiencia global. Durante el periodo analizado, se identificaron varios momentos en los que costos operativos elevados impactaron negativamente en los balances. Por ejemplo:
- El **19 de julio de 2024**, los costos operativos ascendieron a $141,747, contribuyendo a un balance negativo de $-260,383.
- El **15 de julio de 2024**, los costos operativos fueron de $145,435, coincidiendo con un balance negativo de $-43,927.

En contraste, periodos con costos operativos más moderados lograron balances positivos, incluso con ingresos relativamente bajos. Por ejemplo:
- El **28 de septiembre de 2023**, con costos operativos de $90,158, se alcanzó un balance de $400,966.

La optimización de procesos y la reducción de gastos innecesarios son áreas clave de oportunidad para mejorar la eficiencia operativa y aumentar los márgenes de rentabilidad.

---

## 6. **Conclusiones y Recomendaciones**

### Conclusiones
1. Los ingresos y gastos presentan fluctuaciones significativas, con periodos de alto rendimiento intercalados con pérdidas considerables.
2. El balance financiero está fuertemente influenciado por la capacidad de controlar los gastos, especialmente los costos operativos.
3. La relación entre el número de clientes y los resultados financieros no es lineal, lo que sugiere la necesidad de optimizar la generación de ingresos por cliente.
4. Los costos operativos elevados son un factor recurrente en los periodos de pérdida, lo que destaca la importancia de una gestión más eficiente de los recursos.

### Recomendaciones
1. **Control de Costos:** Implementar medidas para reducir los costos operativos, especialmente en periodos de ingresos bajos. Esto podría incluir la renegociación de contratos, la automatización de procesos y la eliminación de gastos innecesarios.
2. **Optimización de Ingresos por Cliente:** Desarrollar estrategias para maximizar los ingresos por cliente, como la oferta de servicios premium o la personalización de productos financieros.
3. **Análisis Predictivo:** Utilizar herramientas de análisis predictivo para anticipar periodos de baja rentabilidad y tomar medidas proactivas para mitigar su impacto.
4. **Diversificación de Ingresos:** Explorar nuevas fuentes de ingresos para reducir la dependencia de los clientes actuales y mejorar la resiliencia financiera.
5. **Gestión de la Base de Clientes:** Priorizar la retención de clientes de alto valor y mejorar la experiencia del cliente para fomentar la lealtad y el crecimiento sostenible.

En resumen, aunque el periodo analizado muestra un desempeño financiero mixto, existen oportunidades claras para mejorar la eficiencia operativa, optimizar los ingresos y fortalecer la estabilidad financiera a largo plazo.

### Podemos ver también el coste que ha tenido la llamada

In [16]:
response.usage.prompt_tokens

5393

In [17]:
response.usage.completion_tokens

1706

In [18]:
def calcular_coste_uso_api(response, modelo: str) -> dict:
    """
    Calcula el costo de uso de la API de Azure OpenAI basado en los tokens usados,
    separando el coste del input (prompt) y del output (completion).

    Para el modelo "gpt-4o":
      - Input: 2.50 $ por millón de tokens.
      - Output: 10.00 $ por millón de tokens.
    (Referencia: https://openai.com/api/pricing/)

    Args:
        response: Objeto de respuesta de la API que contiene el atributo 'usage'.
        modelo (str): Nombre del modelo utilizado (ej: "gpt-4o-mini").

    Returns:
        dict: Diccionario con los costos calculados para entrada, salida y el total.
    """
    #Definimos los precios para el modelo gpt-4o
    if "gpt-4o" in modelo:
        coste_prompt_millon = 2.50 # $ por 1,000,000 tokens de entrada
        coste_completion_millon = 10.00  # $ por 1,000,000 tokens de salida
    else:
        print("Modelo no reconocido.")
        return {}

    #Extraemos los tokens usados del atributo 'usage' del objeto de respuesta (se podría devolver también en el return si se desea)
    prompt_tokens = response.usage.prompt_tokens
    completion_tokens = response.usage.completion_tokens

    #Calculamos el coste para cada parte (teniendo en cuenta que el precio es por millón de tokens)
    coste_prompt = (prompt_tokens / 1000000) * coste_prompt_millon
    coste_completion = (completion_tokens / 1000000) * coste_completion_millon
    coste_total = coste_prompt + coste_completion

    return {
        "coste_prompt": coste_prompt,
        "coste_completion": coste_completion,
        "coste_total": coste_total
    }

# Ejemplo de uso con un objeto de respuesta simulado similar al que describes:
class Usage:
    def __init__(self, prompt_tokens, completion_tokens):
        self.prompt_tokens = prompt_tokens
        self.completion_tokens = completion_tokens

class ChatCompletion:
    def __init__(self, usage):
        self.usage = usage

In [19]:
model = "gpt-4o"
cost = calcular_coste_uso_api(response, model)

print(f"Coste de tokens de entrada (prompt): ${cost['coste_prompt']:.6f}")
print(f"Coste de tokens de salida (completion): ${cost['coste_completion']:.6f}")
print(f"Coste total: ${cost['coste_total']:.6f}")

Coste de tokens de entrada (prompt): $0.013483
Coste de tokens de salida (completion): $0.017060
Coste total: $0.030543


##### - ¿Cómo afecta la estructura del prompt al nivel de detalle y coherencia del informe narrativo generado?
##### - ¿Qué mejoras o ajustes sugerirías para obtener un informe aún más preciso o enfocado en ciertos aspectos?